In [39]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: 

In [45]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

base_model.trainable = False

In [40]:
caminho_dataset = "/content/PetImages.rar"




In [13]:
!apt-get install unrar
!unrar x /content/PetImages.rar -o+


A saída de streaming foi truncada nas últimas 5000 linhas.
Extracting  PetImages/gato/550.jpg                                        81%  OK 
Extracting  PetImages/gato/5500.jpg                                       81%  OK 
Extracting  PetImages/gato/5501.jpg                                       81%  OK 
Extracting  PetImages/gato/5502.jpg                                       81%  OK 
Extracting  PetImages/gato/5503.jpg                                       81%  OK 
Extracting  PetImages/gato/5504.jpg                                       81%  OK 
Extracting  PetImages/gato/5505.jpg                                       81%  OK 
Extracting  PetImages/gato/5506.jpg                                       81%  OK 
Extracting  PetImages/gato/5507.jpg                                       81%  OK 
Extracting  PetImages/gato/5508.jpg                                       81%  OK 
Extracting  PetImages

In [41]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

# Criar geradores de imagens
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = datagen.flow_from_directory(
    "/content/PetImages",
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="training"
)

val_data = datagen.flow_from_directory(
    "/content/PetImages",
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="validation"
)


Found 20000 images belonging to 2 classes.
Found 4998 images belonging to 2 classes.


In [42]:
import os
from PIL import Image

def remover_imagens_corrompidas(diretorio):
    imagens_corrompidas = 0

    for classe in ["cachorro", "gato"]:
        caminho_classe = os.path.join(diretorio, classe)

        for arquivo in os.listdir(caminho_classe):
            caminho_arquivo = os.path.join(caminho_classe, arquivo)

            try:
                img = Image.open(caminho_arquivo)
                img.verify()
            except (IOError, SyntaxError):
                os.remove(caminho_arquivo)
                imagens_corrompidas += 1

    print(f"Total de imagens corrompidas removidas: {imagens_corrompidas}")

remover_imagens_corrompidas("/content/PetImages")


/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Total de imagens corrompidas removidas: 0


In [43]:
# Modelo pré-treinado
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = Flatten()(base_model.output)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1, activation="sigmoid")(x)

modelo = Model(inputs=base_model.input, outputs=x)
modelo.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

modelo.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 112, 112, 32)   │            864 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 112, 112, 32)   │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 112, 112, 32)   │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 112, 112, 32)   │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 112, 112, 16)   │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 112, 112, 16)   │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 112, 112, 96)   │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 112, 112, 96)   │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 112, 112, 96)   │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 113, 113, 96)   │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 56, 56, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├──────────────────────

 Total params: 10,286,401 (39.24 MB)

 Trainable params: 8,028,417 (30.63 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [35]:
import os

def verificar_arquivos_invalidos(diretorio):
    arquivos_invalidos = 0

    for classe in ["cachorro", "gato"]:
        caminho_classe = os.path.join(diretorio, classe)

        for arquivo in os.listdir(caminho_classe):
            caminho_arquivo = os.path.join(caminho_classe, arquivo)

            if not os.path.exists(caminho_arquivo):
                arquivos_invalidos += 1
                print(f"Arquivo ausente: {caminho_arquivo}")

    print(f"Total de arquivos ausentes: {arquivos_invalidos}")

verificar_arquivos_invalidos("/content/PetImages")


Total de arquivos ausentes: 0


In [36]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = datagen.flow_from_directory(
    "/content/PetImages",
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="training"
)

val_data = datagen.flow_from_directory(
    "/content/PetImages",
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="validation"
)


Found 20000 images belonging to 2 classes.
Found 4998 images belonging to 2 classes.


In [52]:
print(train_data.class_indices)
print(train_data.samples)
print(val_data.samples)


{'cachorro': 0, 'gato': 1}
20000
4998


In [60]:
historico = modelo.fit(
    train_data,
    validation_data=val_data,
    epochs=5
)


Epoch 1/5
  2/625 ━━━━━━━━━━━━━━━━━━━━ 19:14 2s/step - accuracy: 0.8438 - loss: 2.5026

KeyboardInterrupt: 

In [30]:
from google.colab import files
uploaded = files.upload()

Saving cat 2.PNG to cat 2.PNG
Saving cat 3.PNG to cat 3.PNG
Saving gat1.PNG to gat1.PNG


In [56]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing import image

pasta_teste = "/content/teste imagens"

arquivos_imagem = [f for f in os.listdir(pasta_teste) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

for arquivo in arquivos_imagem:
    caminho_imagem = os.path.join(pasta_teste, arquivo)

    img = image.load_img(caminho_imagem, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0


    predicao = modelo.predict(img_array)

    classe = "Cachorro" if predicao[0][0] < 0.5 else "Gato"

    print(f"Imagem: {arquivo} -> Predição: {classe}")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Imagem: gat1.PNG -> Predição: Gato
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Imagem: cachorro1.PNG -> Predição: Cachorro
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Imagem: cat 2.PNG -> Predição: Gato
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Imagem: cachorro2.PNG -> Predição: Cachorro
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Imagem: cachorro3.PNG -> Predição: Gato
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Imagem: cat 3.PNG -> Predição: Gato


In [68]:
!git clone https://github.com/SaulMelox/Dio-Transfer-Learning.git


Cloning into 'Dio-Transfer-Learning'...


In [69]:
%cd Dio-Transfer-Learning


/content/Dio-Transfer-Learning


In [73]:
!git add .


In [74]:
!git commit -m "enviando codigo"


A saída de streaming foi truncada nas últimas 5000 linhas.
 create mode 100644 PetImages/gato/5502.jpg
 create mode 100644 PetImages/gato/5503.jpg
 create mode 100644 PetImages/gato/5504.jpg
 create mode 100644 PetImages/gato/5505.jpg
 create mode 100644 PetImages/gato/5506.jpg
 create mode 100644 PetImages/gato/5507.jpg
 create mode 100644 PetImages/gato/5508.jpg
 create mode 100644 PetImages/gato/5509.jpg
 create mode 100644 PetImages/gato/551.jpg
 create mode 100644 PetImages/gato/5510.jpg
 create mode 100644 PetImages/gato/5511.jpg
 create mode 100644 PetImages/gato/5512.jpg
 create mode 100644 PetImages/gato/5513.jpg
 create mode 100644 PetImages/gato/5514.jpg
 create mode 100644 PetImages/gato/5515.jpg
 create mode 100644 PetImages/gato/5516.jpg
 create mode 100644 PetImages/gato/5517.jpg
 create mode 100644 PetImages/gato/5518.jpg
 create mode 100644 PetImages/gato/5519.jpg
 create mode 100644 PetImages/gato/552.jpg
 create mode 100644 PetImages/gato/5520.jpg
 create mode 100644

In [75]:
!git push origin main


fatal: could not read Username for 'https://github.com': No such device or address


In [80]:
!ssh-keygen -t rsa -b 4096 -C "saulorafael9911@gmail.com" -f ~/.ssh/id_rsa -N ""



Generating public/private rsa key pair.
Your identification has been saved in /root/.ssh/id_rsa
Your public key has been saved in /root/.ssh/id_rsa.pub
The key fingerprint is:
SHA256:Bm80JI5Uv28X+cRfmh8TTgFMn3LJ/6KU1AlKYyLc4ZU saulorafael9911@gmail.com
The key's randomart image is:
+---[RSA 4096]----+
|    ..o o ..oo   |
|   . + * oE  .+ o|
|    . = B + .. B |
|       = * o =o.o|
|        S . + =oo|
|       o . . =o++|
|          o + =+o|
|         . o . oo|
|            .   .|
+----[SHA256]-----+


In [81]:
!cat ~/.ssh/id_rsa.pub


ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQCjoVi0gMdxzAj26nGk3apczUOoU2kya8BGDNQyklwDiI/TYbDDIlZligmWFt6TmoWb0DUG1SndOp9dO4icafVLBpcudSVvO9QE5tCjbMDZwDB+lYwSL4CbEe6OW66hMIfhyYVsd0oGzmybCyOpQQpYbxDwACihVdl3bdtAnw3A6oMlFkXaFKMgtLkMFtIoxJjtd8RBY1zUIJjDvBYGZlqX8o0QZAbStckuAKOMoTUGV1I9Df2LsKkO/v2U+nkVWiZm2Y4BECPa5za5JjB7kiKtXrla4xcQ1w6upSgTtUFq619vqa+W+uVBSRXaKxdDzRLM4upUYacf0WpvwaIogkctng7ssCsX46XC1jjSwGgTHdgP0peIBbEbpAzSF7XH1gjA69eHfa0Z20FJT10ez4vcfrSiaNIIQT2OnAzwIlT6s9CRWx9u27Xpn4asOVmliUDNMnYo7pntxPMdRJ8zRnmfc588ZcXMIfxmhOibUgB55ABPBnYCj6GJMJ2fUULPkrev+PXEwI6+0j3kn1BpeNAZtz94XLioJlg1fWF4j8pS//1WDRUS7zQbAAoCR/WdBD4/IveE2MhujYBeNFAavyedNkoQxM9B4muFP4OtVOWNaojztNXYR05K0VUateobH4JzTrYQpGO/RdFobzROm8Psb4JFdKZBId2LYg6D6k36sQ== saulorafael9911@gmail.com


In [82]:
!ssh-keyscan github.com >> ~/.ssh/known_hosts


# github.com:22 SSH-2.0-cb0cff27f
# github.com:22 SSH-2.0-cb0cff27f
# github.com:22 SSH-2.0-cb0cff27f
# github.com:22 SSH-2.0-cb0cff27f
# github.com:22 SSH-2.0-cb0cff27f


In [83]:
!ssh -T git@github.com


Hi SaulMelox! You've successfully authenticated, but GitHub does not provide shell access.


In [87]:
%cd /content
!rm -rf Dio-Transfer-Learning
!git clone git@github.com:SaulMelox/Dio-Transfer-Learning.git
%cd Dio-Transfer-Learning



/content
Cloning into 'Dio-Transfer-Learning'...
/content/Dio-Transfer-Learning


In [88]:
!git add .
!git commit -m "Adicionando código do Colab"
!git push origin main


On branch main

Initial commit

nothing to commit (create/copy files and use "git add" to track)
error: src refspec main does not match any
error: failed to push some refs to 'github.com:SaulMelox/Dio-Transfer-Learning.git'
